When I write '-----------' it means that I am about to write what is not going to go in the script, and when I write '________' it means that I stop writing what is not going to go in the script

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import random
import time
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import undetected_chromedriver as uc
'--------'
import traceback
'_________'


'_________'

In [2]:
browser = webdriver.Chrome() 

In [3]:
page_number = 1

ids_cell_phones = []


'''I execute the infinite while loop that will stop when it no longer detects any more pages 
with products within the loop, mainly the identifier of each cell phone will be obtained and 
it will be stored in ids_cell_phones.'''

while True:

    url = f'https://tienda.personal.com.ar/celulares?page={page_number}'

    browser.get(url)

    time.sleep(random.randint(10,12))

    '''I try to click on a cookie acceptance element if it is present.'''
    try:
        browser.find_element('xpath', '//*[@id="didomi-notice-agree-button"]').click()
    except:
        pass
    
    html = browser.page_source

    soup = bs(html , 'lxml')

    try:
        current_page = int(soup.find('div',{'class':'emsye86z'}).find('a',{'class':'emsye870 emsye871 emsye872'}).text)
    except:
        pass

    if page_number == current_page:
        articles = soup.find('div',{'class': 'emsye86a'})
    else:
        break
    
    page_number += 1

    for article in articles:
        cell_phone_id = article.get('href')

        ids_cell_phones.append(cell_phone_id)

        time.sleep(random.randint(1,3))

        #print(cell_phone_id)


    ids_cell_phones = [phone for phone in ids_cell_phones if phone is not None]

In [4]:
print('Total number of cell phones:', len(ids_cell_phones))

Total number of cell phones: 85


In [40]:


data = {
    'url/id': [],
    'error': []
}

critical_errors = pd.DataFrame(data)
errors_founds = pd.Series()


personal_cell_phones = pd.Series()

'''I define the function parce_cell_phones that takes a cell phone identifier 
as an argument and extracts detailed information about that phone from its individual page.'''

def parse_cell_phones(cell_phone_id):
    
    url = f'https://tienda.personal.com.ar{cell_phone_id}'
    #print('Dirección: ', url)

    browser.get(url)

    time.sleep(random.randint(4,10))

    html = browser.page_source

    soup = bs(html, 'lxml')

    try:
        model = soup.find('h1').text

        price = int(soup.find('div',{'class':'_1qzmwzw89 _1qzmwzw8c _1qzmwzw66 _1qzmwzw6g'})
                    .find('div',{'class':'emsye831'})
                    .text
                    .replace('$','')
                    .replace('.',''))
        
        '''I create the variable more_specifications because i need the ram the characteristics, and i do nor get all the characteristics
        of this variable because this section does not exist in some cell phones, so the ram remains with the value None'''
        
        highlighted_especifications = soup.find('div',{'class': '_1qzmwzw8b _1qzmwzw1k _1qzmwzw6j _1qzmwzw89'}).find_all('div',{'class':'_1qzmwzw89 _1qzmwzw8b _1qzmwzw8c'})

        more_especifications = soup.find('div', {'class': 'emsye84e'}).find_all('div', {'class': '_1qzmwzw89 emsye84j'})
        
    except:
        '-----------'
        errors_founds['url/id'] = url
        errors_founds['error'] = traceback.format_exc()
        df_errors_founds = pd.DataFrame(errors_founds)
        global critical_errors
        critical_errors = pd.concat([critical_errors, df_errors_founds.T])
        '_________'
        return None
    
    ram = None
    camera = None
    processor = None
    storege = None
    screen = None
        
    for div in highlighted_especifications:
        momentary_text = div.find('div',{'class':'emsye846'}).text.lower()
        #print(momentary_text)
        
        if 'procesador' in momentary_text:
            try:
                processor = div.find('div', {'class': 'emsye847'}).text
            except:
                pass

        
        elif 'almacenamiento' in momentary_text:
            try:
                storege = div.find('div', {'class': 'emsye847'}).text.replace('\n',' ')
            except:
                pass


        elif 'cámara' in momentary_text:
            try:
                
                camera = div.find('div', {'class': 'emsye847'}).text
            except:
                pass
        

        elif 'pantalla' in momentary_text:
            try:
                screen = div.find('div', {'class': 'emsye847'}).text
            except:
                pass

        
    for div in more_especifications:
        momentary_text = div.find('div',{'class':'_1qzmwzw89 emsye84k'}).text.lower()
        #print(momentary_text)

        if 'ram' in momentary_text:
            try:
                ram = div.find('span').text
            except:
                pass
    
    '-----------'
    if None in [ram, camera, processor, storege, screen]:
        errors = url
    else:
        errors = None
    '___________'


    personal_cell_phones['model'] = model
    
    personal_cell_phones['price'] = price

    personal_cell_phones['ram'] = ram
    
    personal_cell_phones['processor'] = processor
    
    personal_cell_phones['storege'] = storege
    
    personal_cell_phones['camera'] = camera
    
    personal_cell_phones['screen'] = screen.replace('\m',' ')

    '--------'
    personal_cell_phones['errors'] = errors
    '________'

    df_personal_cell_phones = pd.DataFrame(personal_cell_phones)

    return (df_personal_cell_phones.T)



In [41]:
'''I create a for loop to iterate through the cell phone identifiers, call the pare_cell_phones function with 
each identifier, and add the results to the df_personal_cell_phones DataFrame.'''

df_personal_cell_phones = pd.DataFrame()

for i in range(len(ids_cell_phones)):
    try:
        df_personal_cell_phones = pd.concat([df_personal_cell_phones, parse_cell_phones(ids_cell_phones[i])])
    except:
        pass

    if len(critical_errors) > 10:
            print('Extraction stopped, to many errors')
            break

    print(f'Progress {i+1} of {len(ids_cell_phones)}')

    time.sleep(random.randint(4,8))

#browser.quit()

df_personal_cell_phones.to_csv('celulares_personal.csv', index = False)

print('Number of records obtained: ', len(df_personal_cell_phones))

print('Number of serious errors found: ', len(critical_errors))

print('Number of null data found: ',df_personal_cell_phones.iloc[:,:7].isnull().sum().sum())

print('extraction completed')

Progress 1 of 85
Progress 2 of 85
Progress 3 of 85
Progress 4 of 85
Progress 5 of 85
Progress 6 of 85
Progress 7 of 85
Progress 8 of 85
Progress 9 of 85
Progress 10 of 85
Progress 11 of 85
Progress 12 of 85
Progress 13 of 85
Progress 14 of 85
Progress 15 of 85
Progress 16 of 85
Progress 17 of 85
Progress 18 of 85
Progress 19 of 85
Progress 20 of 85
Progress 21 of 85
Progress 22 of 85
Progress 23 of 85
Progress 24 of 85
Progress 25 of 85
Progress 26 of 85
Progress 27 of 85
Progress 28 of 85
Progress 29 of 85
Progress 30 of 85
Progress 31 of 85
Progress 32 of 85
Progress 33 of 85
Progress 34 of 85
Progress 35 of 85
Progress 36 of 85
Progress 37 of 85
Progress 38 of 85
Progress 39 of 85
Progress 40 of 85
Progress 41 of 85
Progress 42 of 85
Progress 43 of 85
Progress 44 of 85
Progress 45 of 85
Progress 46 of 85
Progress 47 of 85
Progress 48 of 85
Progress 49 of 85
Progress 50 of 85
Progress 51 of 85
Progress 52 of 85
Progress 53 of 85
Progress 54 of 85
Progress 55 of 85
Progress 56 of 85
P

In [42]:
critical_errors

,url/id,error


In [9]:
type(len(critical_errors))

int

In [ ]:
df_personal_cell_phones

In [ ]:
df_personal_cell_phones[df_personal_cell_phones['errors'].notnull()]

In [ ]:
df_personal_cell_phones[df_personal_cell_phones['errors'].notnull()]['errors'].iloc[1]